# quantaTinyLLM Maker
This notebook trains a custom character-level GPT model.

### Instructions:
1.  **Configure your settings** in the next cell.
2.  **Run the Configuration cell.**
3.  **Run the Training cell.** This will mount your Google Drive, load your data, and start training.
4.  Once training is complete, a `checkpoint.pth` file will be saved to your Colab environment.
5.  You can then run the **Inference cell** to see your model generate text, or download the checkpoint to use with `inference.py`.

In [ ]:
# === 1. CONFIGURATION ===
# --- Dataset Settings ---
# This path points to your training data file within your Google Drive.
DATASET_PATH = "/content/drive/MyDrive/MyLLM/training_dataset.txt"
CHECKPOINT_NAME = 'quanta_checkpoint.pth'

# --- Model Hyperparameters ---
# Adjust these to change the size and architecture of your model.
N_EMBD = 128       # The embedding dimension for each token.
N_HEAD = 4         # The number of attention heads.
N_LAYER = 12       # The number of transformer blocks (layers).
BLOCK_SIZE = 128   # The maximum context length for predictions.
DROPOUT = 0.1      # The dropout rate for regularization.

# --- Training Hyperparameters ---
MAX_ITERS = 2000         # Total training iterations.
LEARNING_RATE = 1e-3   # The learning rate for the optimizer.
BATCH_SIZE = 32        # How many sequences to process in parallel.
EVAL_INTERVAL = 100    # How often to evaluate the model and save a checkpoint.

In [ ]:
# === 2. TRAINING ===
# This cell contains all the code to define and train the model.

import math
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from google.colab import drive

# --- Setup ---
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_steps = []
loss_values = []

# --- Data Loading ---
drive.mount('/content/drive')
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    text = f.read()

# --- Tokenizer ---
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# --- Data Splitting ---
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# --- Model Definition ---
class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout, device):
        super().__init__()
        self.device = device
        self.sa = nn.MultiheadAttention(n_embd, n_head, bias=False, device=device, batch_first=True)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        y = self.ln1(x)
        y, _ = self.sa(y, y, y, attn_mask=None, need_weights=False, is_causal=True)
        x = x + y
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout, device):
        super().__init__()
        self.device = device
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size, dropout, device) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, block_size, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# --- Training Class ---
class Trainer:
    def __init__(self, model, optimizer, train_data, val_data):
        self.model = model
        self.optimizer = optimizer
        self.train_data = train_data
        self.val_data = val_data
        self.start_iter = 0
        try:
            print(f"Checking for checkpoint: {CHECKPOINT_NAME}")
            checkpoint = torch.load(CHECKPOINT_NAME, map_location=device)
            if (N_EMBD != checkpoint['n_embd'] or N_HEAD != checkpoint['n_head'] or N_LAYER != checkpoint['n_layer']):
                raise ValueError("Hyperparameter mismatch.")
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.start_iter = checkpoint['iter']
            print(f"Resuming training from iteration {self.start_iter}")
        except FileNotFoundError:
            print("No checkpoint found. Starting training from scratch.")
        except Exception as e:
            print(f"Checkpoint could not be loaded. Starting from scratch. Error: {e}")

    def get_batch(self, split):
        data = self.train_data if split == 'train' else self.val_data
        ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, ))
        x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
        y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
        x, y = x.to(device), y.to(device)
        return x, y

    @torch.no_grad()
    def estimate_loss(self):
        out = {}
        self.model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(EVAL_INTERVAL)
            for k in range(EVAL_INTERVAL):
                X, Y = self.get_batch(split)
                with torch.cuda.amp.autocast():
                    logits, loss = self.model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        self.model.train()
        return out

    def run(self):
        loss_steps.clear()
        loss_values.clear()
        scaler = torch.cuda.amp.GradScaler()
        start_time = time.time()
        for iter in range(self.start_iter, MAX_ITERS):
            if iter > 0 and iter % EVAL_INTERVAL == 0:
                debug_loss = self.estimate_loss()
                elapsed_time = time.time() - start_time
                etc_seconds = (elapsed_time / (iter - self.start_iter + 1)) * (MAX_ITERS - iter)
                print(f"[{elapsed_time:.0f}s] step {iter}: train loss {debug_loss['train']:.4f}, val loss {debug_loss['val']:.4f} (ETC: {etc_seconds/60:.2f} min)")
                checkpoint = {
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'iter': iter,
                    'vocab_size': vocab_size,
                    'itos': itos,
                    'n_embd': N_EMBD,
                    'n_head': N_HEAD,
                    'n_layer': N_LAYER,
                    'block_size': BLOCK_SIZE,
                    'dropout': DROPOUT
                }
                torch.save(checkpoint, CHECKPOINT_NAME)

            xb, yb = self.get_batch('train')
            with torch.cuda.amp.autocast():
                logits, loss = self.model(xb, yb)
            self.optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(self.optimizer)
            scaler.update()

            loss_steps.append(iter)
            loss_values.append(loss.log10().item())

        print(f"Training finished at iteration {MAX_ITERS}.")
        final_checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'iter': MAX_ITERS,
            'vocab_size': vocab_size,
            'itos': itos,
            'n_embd': N_EMBD,
            'n_head': N_HEAD,
            'n_layer': N_LAYER,
            'block_size': BLOCK_SIZE,
            'dropout': DROPOUT
        }
        torch.save(final_checkpoint, CHECKPOINT_NAME)

# --- Execution ---
model = BigramLanguageModel(vocab_size, N_EMBD, N_HEAD, N_LAYER, BLOCK_SIZE, DROPOUT, device).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
trainer = Trainer(model, optimizer, train_data, val_data)
trainer.run()

In [ ]:
# === 3. INFERENCE ===
# This cell can be run independently after a checkpoint has been saved.

import torch
import os

if os.path.exists(CHECKPOINT_NAME):
    print(f"Loading checkpoint '{CHECKPOINT_NAME}' for inference...")
    checkpoint = torch.load(CHECKPOINT_NAME, map_location=device)
    
    # Rebuild the tokenizer from the checkpoint
    itos = checkpoint['itos']
    decode = lambda l: ''.join([itos[i] for i in l])

    # Rebuild the model with the loaded hyperparameters
    inference_model = BigramLanguageModel(
        vocab_size=checkpoint['vocab_size'],
        n_embd=checkpoint['n_embd'],
        n_head=checkpoint['n_head'],
        n_layer=checkpoint['n_layer'],
        block_size=checkpoint['block_size'],
        dropout=checkpoint['dropout'],
        device=device
    )
    m_inference = inference_model.to(device)
    m_inference.load_state_dict(checkpoint['model_state_dict'])
    m_inference.eval()

    print("Model loaded. Generating text...")
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m_inference.generate(context, block_size=checkpoint['block_size'], max_new_tokens=500)[0].tolist()))
else:
    print(f"No checkpoint found named '{CHECKPOINT_NAME}'. Please run the training cell first.")

In [ ]:
# === 4. PLOT LOSS ===
plt.plot(loss_steps, loss_values)